In [1]:
import urllib.request
import zipfile
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator,\
    load_img, img_to_array

In [2]:
url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"

In [3]:
file_name = "horse-or-human.zip"
training_dir = r'horse-or-human/training/'
urllib.request.urlretrieve(url, file_name)

('horse-or-human.zip', <http.client.HTTPMessage at 0x21952933df0>)

In [4]:
validation_file_name = "validation-horse-or-human.zip"
validation_dir = r'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)

('validation-horse-or-human.zip', <http.client.HTTPMessage at 0x2195292e670>)

In [5]:
zip_ref = zipfile.ZipFile(file_name,'r')
zip_ref.extractall(training_dir)
zip_ref.close()

In [6]:
zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [21]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(300,300),
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [22]:
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(300,300),
    class_mode='binary'
)

Found 256 images belonging to 2 classes.


In [23]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 149, 149, 16)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 35, 35, 64)      

In [25]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model.compile(loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy'])

In [26]:
history = model.fit(train_generator, epochs = 15, validation_data=validation_generator)

Epoch 1/15
33/33 [==============================] - 42s 1s/step - loss: 0.7717 - accuracy: 0.5998 - val_loss: 0.6888 - val_accuracy: 0.6641
Epoch 2/15
33/33 [==============================] - 40s 1s/step - loss: 0.6482 - accuracy: 0.6904 - val_loss: 1.1469 - val_accuracy: 0.4961
Epoch 3/15
33/33 [==============================] - 40s 1s/step - loss: 0.5271 - accuracy: 0.7644 - val_loss: 1.9204 - val_accuracy: 0.5039
Epoch 4/15
33/33 [==============================] - 53s 2s/step - loss: 0.4529 - accuracy: 0.7984 - val_loss: 3.9210 - val_accuracy: 0.5000
Epoch 5/15
33/33 [==============================] - 46s 1s/step - loss: 0.4503 - accuracy: 0.8014 - val_loss: 1.1514 - val_accuracy: 0.5117
Epoch 6/15
33/33 [==============================] - 40s 1s/step - loss: 0.3465 - accuracy: 0.8481 - val_loss: 2.2208 - val_accuracy: 0.5078
Epoch 7/15
33/33 [==============================] - 40s 1s/step - loss: 0.3309 - accuracy: 0.8754 - val_loss: 0.7879 - val_accuracy: 0.5859
Epoch 8/15
33/33 [==

In [15]:
img= load_img('./horse-or-human/training/horses/horse01-0.png', target_size=(300,300))
x= img_to_array(img)
x= np.expand_dims(x,axis=0)

In [20]:
image_tensor = np.vstack([x])
classes = model.predict(image_tensor)

if (classes[0] > 0.5):
    print("Human")
else:
    print("Horse")

Horse
